# Material Complementar da Aula 15

In [ ]:
import networkx as nx
import freeman as fm

from math import isclose
from random import choice, shuffle

Vamos seguir um modelo simplificado, no qual um empreendedor distribui tempo e recursos de forma igual dentre suas conexões.

O **investimento** de tempo e recursos que $n$ dedica a sua relação com $m$ é

$$i_{nm} = \left\{\begin{array}{ll}
\frac{1}{d_n} & \textrm{se existe aresta entre }n\textrm{ e }m;\\
            0 & \textrm{caso contrário},\\
\end{array}\right.$$

onde $d_n$ é o degree de $n$.

In [ ]:
def investment(g, n, m):
    if g.has_edge(n, m):
        return 1 / g.degree(n)
    return 0

Vamos propor uma métrica que um empreendedor busque *reduzir* para conseguir lacunas estruturais.

A **restrição local** de $n$ em relação a $m$ é

$$c_{nm} = \left(i_{nm} + \sum_{k \in V(n)} i_{nk} \cdot i_{km}\right)^2,$$

onde $V(n)$ é o conjunto dos vizinhos de $n$.

Como vimos anteriormente que $i_{nm} = 1/d_n$, podemos observar que a parte esquerda dessa fórmula é reduzida por degrees maiores. Por outro lado, a somatória da parte direita é aumentada por conexões *redundantes*, ou seja, nós que não precisam ser seus vizinhos porque já são vizinhos-de-vizinhos.

Assim, pode-se dizer que a redução dessa métrica busca encontrar um equilíbrio entre *degree alto* e *redundância baixa*.

In [ ]:
 def local_constraint(g, n, m):
    i = investment(g, n, m)
    for k in g.neighbors(n):
        i += investment(g, n, k) * investment(g, k, m)
    return i**2

A **restrição** de $n$, a princípio, deveria ser a somatória de todas as restrições locais. No entanto, essa definição tem algo estranho: ela é brutalmente minimizada (vale zero) pela situação em que $n$ não tem vizinhos. Como essa situação não parece ser desejável para um empreendedor (aí já é lacuna demais!), vamos fazer uma pequena gambiarra:

$$C_n = \left\{\begin{array}{ll}
                       2 & \textrm{se o degree de }n\textrm{ é zero};\\
\sum_{m \in V(n)} c_{nm} & \textrm{caso contrário}.\\
\end{array}\right.$$

In [ ]:
def constraint(g, n):
    if g.degree(n) == 0:
        return 2
    c = 0
    for m in g.neighbors(n):
        c += local_constraint(g, n, m)
    return c

Para verificar se essa proposta de métrica funciona, vamos considerar um exemplo simples.

In [ ]:
g = fm.load('dados/exercicios5/buskens-vanderijt.gml')
g.label_nodes()
g.draw()

Vamos calcular a restrição de cada nó e armazenar no *dataframe* interno do grafo.

In [ ]:
g.set_nodedata('constraint', lambda n: constraint(g, n))
g.nodeframe

Essa situação (estrela) é ideal para A em termos de intermediação, pois todos dependem dele para contatar outros. Isso é confirmado pelos resultados da fórmula, que indicam uma restrição baixa para A e alta para os outros.

Agora vamos ver o que acontece quando "flipamos" algumas dessas relações, ou seja, adicionamos ou removemos arestas.

Em particular, vamos ver o que acontece quando flipamos a relação entre B e C.

In [ ]:
g.switch('B', 'C')
g.draw()

Claramente, A ficou menos broker e B e C ficaram um pouquinho mais brokers, ainda que não muito porque suas conexões são redundantes. Vamos ver qual foi o *impacto* dessa flipagem nas restrições.

In [ ]:
g.set_nodedata('constraint', lambda n: constraint(g, n))
g.nodeframe

Pois é, a restrição de A continua melhor que a dos outros, mas quase dobrou!

Vamos resolver isso removendo a aresta entre A e C, que é redundante.

In [ ]:
g.switch('A', 'C')
g.draw()

Isso eliminou a redundância e também deixou B mais broker do que antes. C, por outro lado, voltou a ser totalmente não-broker.

Vejamos se a métrica está coerente com todas essas observações.

In [ ]:
g.set_nodedata('constraint', lambda n: constraint(g, n))
g.nodeframe

Ok, mas o que vai acontecer com a rede se *todos* querem lacunas estruturais?

Para verificar isso, vamos fazer uma *modelagem baseada em agentes*. A bilioteca da disciplina possui uma classe `Simulation` que pode ser subclasseada para facilitar a construção de simulações. Tente entender o código abaixo com calma, pois é algo que pode ser útil para seu projeto.

In [ ]:
class HoleSimulation(fm.Simulation):


    # O construtor pode ser como você quiser e o esperado é que
    # seja usado para receber os parâmetros principais da
    # simulação. No caso, o único parâmetro é o número de nós.
    def __init__(self, num_nodes):
        self.num_nodes = num_nodes


    # Você PODE sobrescrever este método para especificar o que
    # deve ser feito no início da simulação. No caso, construímos
    # o grafo. Por simplicidade, é um aleatório com densidade 50%.
    def before_each(self):
        self.g = fm.Graph(nx.erdos_renyi_graph(self.num_nodes, 0.5))


    # Você DEVE sobrescrever este método para especificar o que
    # deve ser feito em cada iteração da simulação. E esse método
    # DEVE devolver um booleano indicando se a simulação deve
    # continuar ou não. Se for para parar, deve devolver False.
    def iterate(self):

        # Vamos criar uma variável local para deixar o código menos
        # carregado. Sem isso, seria muito self para lá e para cá.
        g = self.g

        # Cache de restrições para evitar recálculos. Chaves são nós
        # e valores são os respectivos resultados da função constraint.
        constraints = {}

        # Para ter o efeito de escolher nós aleatoriamente e sem
        # reposição, simplesmente embaralhamos o conjunto original
        # e fazemos um loop nesses conjunto de nós embaralhados.
        nodes = list(g.nodes)
        shuffle(nodes)
        for n in nodes:

            # Calcula a restrição de n se não estiver no cache.
            if n not in constraints:
                constraints[n] = constraint(g, n)

            # Vamos descobrir, para cada m diferente de n, o impacto
            # que n sofreria em sua restrição se flipasse a relação
            # com m. Esses impactos são todos armazenados em um
            # dicionário: chaves são nós e valores são os impactos.
            impacts = {}
            for m in nodes:
                if n != m:

                    # Calcula a restrição de m se não estiver no cache.
                    if m not in constraints:
                        constraints[m] = constraint(g, m)

                    # Flipa a relação de n com m.
                    g.switch(n, m)

                    # Se a flipagem criou a relação e a restrição de m
                    # aumentou por causa disso, então a flipagem não é
                    # válida, pois a criação de uma relação depende da
                    # vontade de ambos. Para que isso seja devidamente
                    # desconsiderado, definimos o impacto como zero.
                    if g.has_edge(n, m) and constraint(g, m) > constraints[m]:
                        impacts[m] = 0

                    # Senão, o cálculo de impacto é direto: restrição
                    # pós-flipagem menos a restrição pré-flipagem.
                    else:
                        impacts[m] = constraint(g, n) - constraints[n]

                    # Depois de calcular o impacto, precisamos anular a
                    # flipagem, pois a fizemos apenas para ver o impacto.
                    g.switch(n, m)

            # Agora que descobrimos o impacto relacionado a cada m,
            # vamos descobrir qual foi o menor impacto de todos.
            lowest = min(value for value in impacts.values())

            # Se o menor impacto for negativo, então sabemos que
            # pelo menos uma das flipagens reduziu a restrição.
            if lowest < 0:

                # Vamos recuperar o m responsável por esse menor
                # impacto. Se houver empate, a escolha é aleatória.
                m = choice([m for m in impacts if isclose(impacts[m], lowest)])

                # Refazemos a flipagem de m, agora definitivamente.
                g.switch(n, m)

                # Como uma flipagem foi aceita, a simulação continua.
                return True

        # Se nenhuma flipagem foi aceita, a simulação termina.
        return False


    # Você PODE sobrescrever este método para especificar o que
    # deve ser feito no final da simulação. No caso, adicionamos
    # o betweenness de cada nó ao dataframe interno da simulação.
    def after_each(self, repetition, iterations, elapsed):
        g = self.g

        b = nx.betweenness_centrality(g)

        for n in g.nodes:

            # O método append da simulação adiciona os dados que
            # você quiser ao dataframe interno da simulação. Você
            # deve passar um dicionário, pois é necessário dizer
            # qual é a coluna em que deve ser colocado cada dado.
            self.append({
                'id': n,
                'betweenness': b[n],
            })

Para rodar a simulação, basta instanciar um objeto da classe e rodar o método `run`. Esse método devolve o dataframe interno da simulação. Se você nunca chamou `self.append`, ele estará vazio.

In [ ]:
NUM_NODES = 15

s = HoleSimulation(NUM_NODES)

s.run()

Como podem ver, os betweennesses estão uniformemente baixos? Porque? Vamos visualizar a rede no final da simulação, usando o algoritmo de layout `spring`, que busca aproximar grupos *coesos* (com muitas arestas entre si).

In [ ]:
g = fm.Graph(s.g)
g.move('spring')
g.draw()

Hmmm, não parece haver nada de especial...

Então vamos tentar outra coisa: o método `move_complement` é análogo ao `move`, mas "finge" que o grafo é na verdade seu *complemento*. Em outras palavras, ele finge que arestas são não-arestas e vice-versa. Do ponto de vista do algoritmo `spring`, isso significa que ele passa a aproximar grupos *independentes* (com poucas arestas entre si).

In [ ]:
g.move_complement('spring')
g.draw()

E, agora sim, está evidente que o grafo ficou *bipartido* (two-mode).

Continua no notebook *exercicios5.ipynb*.